In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


<h2><b> SMILES tokenization assesses the Tokenizer subsystem on unconditional generation of SMILES and assesses the validity and Tanimoto similarity values. </b></h2>

---

# Install Libraries

In [ ]:
!pip install SmilesPE langchain ctransformers torch rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 13.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.

In [ ]:
from transformers import BertTokenizerFast, BertLMHeadModel
checkpoint = 'unikei/bert-base-smiles'
bert_tokenizer = BertTokenizerFast.from_pretrained(checkpoint)
bert_model = BertLMHeadModel.from_pretrained(checkpoint)

# Initialize empty lists to store encoded inputs
input_ids_list = []
attention_mask_list = []

vocab_file_path = '/content/drive/MyDrive/LLama2HealthCareChatBot-master/SPE_ChEMBL.txt'

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/306k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


# Default SMILESPE Tokenizer

In [ ]:
# SMILESPE Fragment Level Tokenizer
import codecs
from SmilesPE.tokenizer import *

def test_spe():
    vocab_file_path = '/content/drive/MyDrive/LLama2HealthCareChatBot-master/SPE_ChEMBL.txt'
    spe_vob= codecs.open(vocab_file_path)
    spe = SPE_Tokenizer(spe_vob)

    smi = 'CC(=O)Oc1ccccc1C(=O)O'
    smiles = spe.tokenize(smi)

    return smiles

# Update BERT Vocab List

In [ ]:
def update_tokens(tokeniz, modell, vocab_file_path):
  # Update BERT tokens
  tokeniz.add_tokens(vocab_file_path)

  # Save the updated tokenizer
  tokeniz.save_pretrained(checkpoint)

  # If you modified the vocabulary file, make sure to load it in the model
  modell.resize_token_embeddings(len(tokeniz))

  # Save updated model
  modell.save_pretrained(checkpoint)

  return tokeniz, modell

# Chemical Validity Check Function (using RDKit)

In [ ]:
from rdkit import Chem

def chemical_validity_check(smi, file_to_edit):
  m = Chem.MolFromSmiles(smi, sanitize=False)
  if m is None:
    print('invalid', file=file_to_edit)
    return 0
  else:
    print('valid', file=file_to_edit)
    return 1

# Perform Mask Tokenization



In [ ]:
from transformers import BertLMHeadModel, BertTokenizerFast
import torch
from rdkit import Chem
from rdkit.Chem import AllChem
import random

def calculate_tanimoto_similarity(smiles1, smiles2):
    # Convert SMILES to RDKit molecules
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)

    # Calculate Morgan fingerprints
    fp1 = AllChem.GetMorganFingerprint(mol1, 2)
    fp2 = AllChem.GetMorganFingerprint(mol2, 2)

    # Calculate Tanimoto similarity
    similarity = AllChem.DataStructs.TanimotoSimilarity(fp1, fp2)

    return similarity

def mask_tokenization_process_default(smiles, maximum_count, output_file):
    checkpoint = 'unikei/bert-base-smiles'
    tokenizer = BertTokenizerFast.from_pretrained(checkpoint)
    model = BertLMHeadModel.from_pretrained(checkpoint, is_decoder=True)

    vocab_file_path = '/content/drive/MyDrive/LLama2HealthCareChatBot-master/SPE_ChEMBL.txt'
    spe_vob= codecs.open(vocab_file_path)
    spe = SPE_Tokenizer(spe_vob)

    tokenizer, model = update_tokens(tokenizer, model, vocab_file_path)

    output = []

    print("SMILES Before:", smiles, file=output_file)

    # Tokenize the input
    input_ids = tokenizer.encode(smiles, return_tensors="pt")

    valid = False

    # Set max_count threshold of possible generations to avoid infinite loop
    actual_count = 0

    while (actual_count != maximum_count):  # Generate 1 sequence
        # Generate sequence
        output_sequences = model.generate(
            input_ids,
            max_new_tokens=1,
            do_sample=True,  # Enable sampling (to allow non-deterministic SMILES completion)
            pad_token_id=tokenizer.eos_token, # May have to use better fragment library with pairwise bracket
            output_hidden_states=True
        )

        # Decode the sequence
        decoded_sequence = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

        # Extract the last token (to replace [MASK])
        token = output_sequences[0][-1].item()
        token_str = tokenizer.decode(token)

        # Tokenize input SMILES
        smi = spe.tokenize(smiles)

        tokenized_string = smi.split()
        token_index = random.randint(0, len(tokenized_string) - 1)
        token_to_replace = tokenized_string[token_index]
        print("Token To Replace:", token_to_replace, file=output_file)
        print("Tokenized String:", tokenized_string, file=output_file)

        # Replace the [MASK] token with the predicted token
        decoded_sequence = decoded_sequence.replace(token_to_replace, token_str)

        x = chemical_validity_check(decoded_sequence.replace(" ", ""), output_file)
        similarity = -1
        if x == 1:
            valid = True
            if (Chem.MolFromSmiles(decoded_sequence.replace(" ", "")) != None):
              similarity = calculate_tanimoto_similarity(smiles, decoded_sequence.replace(" ", ""))
        else:
            valid = False

        print("SMILES After: ", decoded_sequence, file=output_file)
        # Calculate the score for the generated sequence
        with torch.no_grad():
            logits = model.forward(input_ids=output_sequences, return_dict=True).logits
            softmax_scores = torch.softmax(logits[0, -1, :], dim=-1)  # Applying softmax to get probabilities
        score = softmax_scores.max().item()  # Taking the maximum probability as the score

        # Append sequence information to output
        output.append({
            "sequence": decoded_sequence,
            'validity': "valid" if valid else "invalid",
            'similarity': similarity,
            "score": score,
        })

        actual_count += 1

    return output

## Generate pipeline to benchmark performance

In [ ]:
import pandas as pd
import statistics

def assess_model_validity_similarity(validity_list, similarity_list):
    validity = output['validity']
    similarity = output['similarity']

def run_model_visualize(num_samples, maximum_count, output_file):
    train_dataset = pd.read_csv('/content/drive/MyDrive/LLama2HealthCareChatBot-master/data/BindingAffinity_data/binding_affinity_train.csv')
    smiles_list = list(train_dataset['Canonical SMILE'])

    validity_count = 0

    validity_list = []
    similarity_list = []

    with open(output_file, "a") as output_file:
      for i in range(num_samples):
          smiles = random.choice(smiles_list)
          output_list = mask_tokenization_process_default(smiles, maximum_count, output_file)
          print(output_list)
          for val in output_list:
            validity = val['validity']
            similarity = val['similarity']

            if (validity == 'valid'):
              validity_count += 1

            if similarity != -1:
                similarity_list.append(similarity)


          validity_list.append(validity_count/maximum_count)

    avg_validity = 0
    avg_similarity = 0
    print(validity_list)
    print(similarity_list)
    if (validity_list):
      avg_validity = statistics.mean(validity_list)
    if (similarity_list):
      avg_similarity = statistics.mean(similarity_list)

    return avg_validity, avg_similarity

## Perform assessment of 100 SMILES generation from each of 20 random SMILES

In [ ]:
validity, similarity = run_model_visualize(20, 100, 'output.txt')

[19:46:11] SMILES Parse Error: syntax error while parsing: ((1=((2((((3=(N((4=((=((2=(34)((()()N((1)((
[19:46:11] SMILES Parse Error: Failed parsing SMILES '((1=((2((((3=(N((4=((=((2=(34)((()()N((1)((' for input: '((1=((2((((3=(N((4=((=((2=(34)((()()N((1)(('
[19:46:12] SMILES Parse Error: syntax error while parsing: CC1=CC2C(CC3=CN(C4=CC=CC2=C34)C(C)C)N(C1)C(
[19:46:12] SMILES Parse Error: Failed parsing SMILES 'CC1=CC2C(CC3=CN(C4=CC=CC2=C34)C(C)C)N(C1)C(' for input: 'CC1=CC2C(CC3=CN(C4=CC=CC2=C34)C(C)C)N(C1)C('
[19:46:13] SMILES Parse Error: syntax error while parsing: CC1=CC2C(CC3=CN(C4=CC=CC2=C34)C(C)C)N(C1)C(
[19:46:13] SMILES Parse Error: Failed parsing SMILES 'CC1=CC2C(CC3=CN(C4=CC=CC2=C34)C(C)C)N(C1)C(' for input: 'CC1=CC2C(CC3=CN(C4=CC=CC2=C34)C(C)C)N(C1)C('
[19:46:13] SMILES Parse Error: syntax error while parsing: CC1=(C(CC3=CN(C4=CC=(=C34)C(C)C)N(C1)C(
[19:46:13] SMILES Parse Error: Failed parsing SMILES 'CC1=(C(CC3=CN(C4=CC=(=C34)C(C)C)N(C1)C(' for input: 'CC1=(C(CC3=CN(C4=

[{'sequence': '((1 = ((2( ( ((3 = (N ( (4 = (( = ((2 = (34 ) ( ( ( ) ( ) N ( (1 ) ( (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999562501907349}, {'sequence': 'CC1 = CC2C ( CC3 = CN ( C4 = CC = CC2 = C34 ) C ( C ) C ) N ( C1 ) C (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999562501907349}, {'sequence': 'CC1 = CC2C ( CC3 = CN ( C4 = CC = CC2 = C34 ) C ( C ) C ) N ( C1 ) C (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999562501907349}, {'sequence': 'CC1 = (C ( CC3 = CN ( C4 = CC = ( = C34 ) C ( C ) C ) N ( C1 ) C (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999562501907349}, {'sequence': 'CC1 = (C ( CC3 = CN ( C4 = CC = ( = C34 ) C ( C ) C ) N ( C1 ) C (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999562501907349}, {'sequence': 'CC1 = CC2C ( CC3 = CN ( C4 = CC = CC2 = C34 ) C ( C ) C ) N ( C1 ) C (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999562501907349}, {'sequence': 'CC1 = CC2C ( CC3 = CN ( C4 = CC = CC2 = C34 )

[19:47:00] SMILES Parse Error: syntax error while parsing: CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=CNC3=NC=C(C=C23)Cl)F(
[19:47:00] SMILES Parse Error: Failed parsing SMILES 'CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=CNC3=NC=C(C=C23)Cl)F(' for input: 'CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=CNC3=NC=C(C=C23)Cl)F('
[19:47:00] SMILES Parse Error: syntax error while parsing: CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=CNC3=NC=C(C=C23)Cl)F(
[19:47:00] SMILES Parse Error: Failed parsing SMILES 'CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=CNC3=NC=C(C=C23)Cl)F(' for input: 'CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=CNC3=NC=C(C=C23)Cl)F('
[19:47:01] SMILES Parse Error: syntax error while parsing: CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=(C3=NC=C(C=C23)Cl)F(
[19:47:01] SMILES Parse Error: Failed parsing SMILES 'CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=(C3=NC=C(C=C23)Cl)F(' for input: 'CCCS(=O)(=O)NC1=C(C(=C(C=C1)F)C(=O)C2=(C3=NC=C(C=C23)Cl)F('
[19:47:01] SMILES Parse Error: syntax error while parsing: CCCS(=O)(=O)NC1=C(C(=C(

[{'sequence': 'CCCS ( = O ) ( = O ) NC1 = C ( C ( = C ( C = C1 ) F ) C ( = O ) C2 = CNC3 = NC = C ( C = C23 ) Cl ) F (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9558146595954895}, {'sequence': 'CCCS ( = O ) ( = O ) NC1 = C ( C ( = C ( C = C1 ) F ) C ( = O ) C2 = CNC3 = NC = C ( C = C23 ) Cl ) F (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9558146595954895}, {'sequence': 'CCCS ( = O ) ( = O ) NC1 = C ( C ( = C ( C = C1 ) F ) C ( = O ) C2 = (C3 = NC = C ( C = C23 ) Cl ) F (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9558146595954895}, {'sequence': 'CCCS ( = O ) ( = O ) NC1 = C ( C ( = C ( C = C1 ) F ) C ( = O ) C2 = CNC3 = NC = C ( C = C23 ) Cl ) F (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9558146595954895}, {'sequence': 'CCCS ( = O ) ( = O ) NC1 = C ( C ( = C ( C = C1 ) F ) C ( = O ) C2 = (C3 = NC = C ( C = C23 ) Cl ) F (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9558146595954895}, {'sequence': 'CCCS ( = O ) ( = O ) NC1 = C ( C

[19:48:00] SMILES Parse Error: syntax error while parsing: C1CCC(C1)C(CC#N)N2C=C(C=N2)C3=C4C=CNC4=NC=N3(
[19:48:00] SMILES Parse Error: Failed parsing SMILES 'C1CCC(C1)C(CC#N)N2C=C(C=N2)C3=C4C=CNC4=NC=N3(' for input: 'C1CCC(C1)C(CC#N)N2C=C(C=N2)C3=C4C=CNC4=NC=N3('
[19:48:01] SMILES Parse Error: syntax error while parsing: C1CCC(C1)C(CC#N)N2C=C(C=N2)C3=C4C=(C4=NC=N3(
[19:48:01] SMILES Parse Error: Failed parsing SMILES 'C1CCC(C1)C(CC#N)N2C=C(C=N2)C3=C4C=(C4=NC=N3(' for input: 'C1CCC(C1)C(CC#N)N2C=C(C=N2)C3=C4C=(C4=NC=N3('
[19:48:01] SMILES Parse Error: syntax error while parsing: C1CCC(C1)C(CC#N)(C=C(C=()C3=C4C=CNC4=NC=N3(
[19:48:01] SMILES Parse Error: Failed parsing SMILES 'C1CCC(C1)C(CC#N)(C=C(C=()C3=C4C=CNC4=NC=N3(' for input: 'C1CCC(C1)C(CC#N)(C=C(C=()C3=C4C=CNC4=NC=N3('
[19:48:02] SMILES Parse Error: syntax error while parsing: C1CCC(C1)C(CC#N)(C=C(C=()C3=C4C=CNC4=NC=N3(
[19:48:02] SMILES Parse Error: Failed parsing SMILES 'C1CCC(C1)C(CC#N)(C=C(C=()C3=C4C=CNC4=NC=N3(' for input: '

...

[{'sequence': 'CCC( ( CC1 = CC = C ( C = C1 ) C2 = CC = CC = C2C3 = ((( = (3 ) C4 = (C = (C5 = C4( = C(5CC ( = O ) O (', 'validity': 'invalid', 'similarity': -1, 'score': 0.999954104423523}, {'sequence': 'CCCN ( CC1 = CC = C ( C = C1 ) C2 = CC = CC = C2C3 = NNN = N3 ) C4 = NC = NC5 = C4N = CN5CC ( = O ) O (', 'validity': 'invalid', 'similarity': -1, 'score': 0.999954104423523}, {'sequence': 'CC( ( CC1 = CC = C ( C = C1 ) C2 = CC = CC = C2C3 = NNN = N3 ) C4 = NC = NC5 = C4N = (5CC ( = O ) O (', 'validity': 'invalid', 'similarity': -1, 'score': 0.999954104423523}, {'sequence': 'CCCN ( CC1 = CC = C ( C = C1 ) C2 = CC = CC = C2C3 = NNN = N3 ) C4 = NC = NC5 = C4N = CN5CC ( = O ) O (', 'validity': 'invalid', 'similarity': -1, 'score': 0.999954104423523}, {'sequence': 'CCCN ( CC1 = CC = C ( C = C1 ) C2 = CC = CC = C2C3 = NNN = N3 ) C4 = NC = NC( = C4N = CN(CC ( = O ) O (', 'validity': 'invalid', 'similarity': -1, 'score': 0.999954104423523}, {'sequence': 'CCCN ( CC1 = CC = C ( C = C1 ) C2 = C

[20:02:54] SMILES Parse Error: unclosed ring for input: 'CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)NC(=C3C4=C(C=C(C=C4)C(=O)OC)NC3=O)C5=CC=CC=C51'
[20:02:55] SMILES Parse Error: extra close parentheses while parsing: CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)NC(=C3C4=C(C=C(C=C4)C(=O)OC)NC3=O)C5=CC=CC=C5)
[20:02:55] SMILES Parse Error: Failed parsing SMILES 'CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)NC(=C3C4=C(C=C(C=C4)C(=O)OC)NC3=O)C5=CC=CC=C5)' for input: 'CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)NC(=C3C4=C(C=C(C=C4)C(=O)OC)NC3=O)C5=CC=CC=C5)'
[20:02:56] SMILES Parse Error: syntax error while parsing: CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)NC(=C3C4=C(C=C(C=C4)C(=O)OC)NC3=O)C5=CC=CC=C5=
[20:02:56] SMILES Parse Error: Failed parsing SMILES 'CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)NC(=C3C4=C(C=C(C=C4)C(=O)OC)NC3=O)C5=CC=CC=C5=' for input: 'CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)NC(=C3C4=C(C=C(C=C4)C(=O)OC)NC3=O)C5=CC=CC=C5='
[20:02:56] SMILES Parse Error: syntax error while parsing: CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)NC(=C3C4=C(C=C(C=

[{'sequence': 'CN1CCN ( CC1 ) CC ( = O ) N ( C ) C2 = CC = C ( C = C2 ) NC ( = C3C4 = C ( C = C ( C = C4 ) C ( = O ) OC ) NC3 = O ) C5 = CC = CC = C51', 'validity': 'invalid', 'similarity': -1, 'score': 0.14231126010417938}, {'sequence': 'CN1CCN ( CC1 ) CC ( = O ) N ( C ) C2 = CC = C ( C = C2 ) NC ( = C3C4 = C ( C = C ( C = C4 ) C ( = O ) OC ) NC3 = O ) C5 = CC = CC = C5 )', 'validity': 'invalid', 'similarity': -1, 'score': 0.9672983884811401}, {'sequence': 'CN1CCN ( CC1 ) CC ( = O ) N ( C ) C2 = CC = C ( C = C2 ) NC ( = C3C4 = C ( C = C ( C = C4 ) C ( = O ) OC ) NC3 = O ) C5 = CC = CC = C5 =', 'validity': 'invalid', 'similarity': -1, 'score': 0.9884144067764282}, {'sequence': 'CN1CCN ( CC1 ) CC ( = O ) N ( C ) C2 = CC = C ( C = C2 ) NC ( = C3C4 = C ( C = C ( C = C4 ) C ( = O ) OC ) NC3 = O ) = = CC = CC = = =', 'validity': 'invalid', 'similarity': -1, 'score': 0.9884144067764282}, {'sequence': 'CN1CCN ( CC1 ) CC ( = O ) N ( C ) C2 = CC = C ( C = C2 ) NC ( = C3C4 = C ( C = C ( C = C4 )

[20:04:23] SMILES Parse Error: syntax error while parsing: C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2(
[20:04:23] SMILES Parse Error: Failed parsing SMILES 'C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2(' for input: 'C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2('
[20:04:24] SMILES Parse Error: syntax error while parsing: C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2(
[20:04:24] SMILES Parse Error: Failed parsing SMILES 'C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2(' for input: 'C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2('
[20:04:25] SMILES Parse Error: syntax error while parsing: C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2(
[20:04:25] SMILES Parse Error: Failed parsing SMILES 'C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2(' for input: 'C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2('
[20:04:25] SMILES Parse Error: syntax error while parsing: C1=CC=C(C=C1)C2=NC(=C3C=CC(=O)C=C3)C(=C4C=CNC=C4)N2(
[20:04:25] SMILES Parse Error:

[{'sequence': 'C1 = CC = C ( C = C1 ) C2 = NC ( = C3C = CC ( = O ) C = C3 ) C ( = C4C = CNC = C4 ) N2 (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999635219573975}, {'sequence': 'C1 = CC = C ( C = C1 ) C2 = NC ( = C3C = CC ( = O ) C = C3 ) C ( = C4C = CNC = C4 ) N2 (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999635219573975}, {'sequence': 'C1 = CC = C ( C = C1 ) C2 = NC ( = C3C = CC ( = O ) C = C3 ) C ( = C4C = CNC = C4 ) N2 (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999635219573975}, {'sequence': 'C1 = CC = C ( C = C1 ) C2 = NC ( = C3C = CC ( = O ) C = C3 ) C ( = C4C = CNC = C4 ) N2 (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999635219573975}, {'sequence': 'C1 = CC = C ( C = C1 ) C2 = NC ( = C3C = CC ( = O ) C = C3 ) C ( = C4C = CNC = C4 ) N2 (', 'validity': 'invalid', 'similarity': -1, 'score': 0.9999635219573975}, {'sequence': 'C1 = CC = C ( C = C1 ) C2 = NC ( = C3C = CC ( = O ) C = C3 ) C ( = C4C = CNC = C4 ) ( (', 'validity': 'in

## Observe Results

In [ ]:
print("Validity Percentage:", validity*100)
print("Similarity Percentage: ", similarity*100)

# For 20 SMILES, each with 100 generations, we obtain validity of 0.3905 and similarity of 0.82891.

Validity Percentage: 27.200000000000003
Similarity Percentage:  78.49713607308159


In [ ]:
with open('output.txt', "a") as output_file:
    print("Average Validity: ", validity, file=output_file)
    print("Average Similarity: ", similarity, file=output_file)

# Additional Notes and Comments

> Could Explore SEFLIES tokenization as Future Development

In [1]:
# EXTRA NOTES ON IMPLEMENTATION OF PROJECT
"""
                       Inpainting              |            Generation
                       (edit)

Unconditional          [Validity, Similarity]  |        [Validity, Similarity]

Conditional            [Success]               |            {Success}

We can move anticlockwise from the top right quadrant in shaping the decision-making process / direction for the thesis
At each step, you highlight challenges (representing proteins, making Q&A prompts, limitations on token size, finding educational assessments that fit college-level chemistry questions).

Thesis Format
- Results and discussion of results could be a section of each chapter
- Each subsection breaks down the results and the storyline flows (like the research discovery process, the challenges faced)

Example: orphaned parentheses does not satisfy my style of inpainting. This is a challenge. To solve this partially, we explored SELFIES. There are benefits with SELFIES ...
         If SELFIES does not work, you don't have to find a solution depending on time constraints. You can say there needs to be more work done to improve small molecule representation by ... and leave to other.
"""

IndentationError: unexpected indent (<ipython-input-1-d553d348becf>, line 3)